In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
!pip install wandb
import wandb
wandb.login()



     |████████████████████████████████| 1.7 MB 9.9 MB/s 
     |████████████████████████████████| 139 kB 40.2 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 180 kB 41.9 MB/s 
     |████████████████████████████████| 296 kB 43.6 MB/s 
     |████████████████████████████████| 63 kB 3.4 MB/s 
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=1263fe295e31248a2717215c9f1578140f3452c21d083a5d257c974f8dc2c355
  Stored in directory: /root/.cache/pip/wheels/29/93/c6/762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=45a825588ad7657e5bd07fbeb02e9250dc5fc42785f384822767c64b2f676c51
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=9e51735373341e26af534ecfc033c7a5e90b9e8944

KeyboardInterrupt: Interrupted by user

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
file = open("dffe.txt").read()

In [ ]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [ ]:
processed_inputs = tokenize_words(file)


In [ ]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [ ]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 145075
Total vocab: 65


In [ ]:
seq_length = 100
x_data = []
y_data = []

In [ ]:
for i in range(0, input_len - seq_length, 1):
     in_seq = processed_inputs[i:i + seq_length]
     out_seq = processed_inputs[i + seq_length]
     x_data.append([char_to_num[char] for char in in_seq])
     y_data.append(char_to_num[out_seq])


In [ ]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)


Total Patterns: 144975


In [ ]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [ ]:
y = tensorflow.keras.utils.to_categorical(y_data)

In [ ]:
X

array([[[0.24615385],
        [0.23076923],
        [0.23076923],
        ...,
        [0.33846154],
        [0.16923077],
        [0.44615385]],

       [[0.23076923],
        [0.23076923],
        [0.33846154],
        ...,
        [0.16923077],
        [0.44615385],
        [0.44615385]],

       [[0.23076923],
        [0.33846154],
        [0.        ],
        ...,
        [0.44615385],
        [0.44615385],
        [0.        ]],

       ...,

       [[0.43076923],
        [0.16923077],
        [0.53846154],
        ...,
        [0.        ],
        [0.24615385],
        [0.16923077]],

       [[0.16923077],
        [0.53846154],
        [0.23076923],
        ...,
        [0.24615385],
        [0.16923077],
        [0.29230769]],

       [[0.53846154],
        [0.23076923],
        [0.43076923],
        ...,
        [0.16923077],
        [0.29230769],
        [0.46153846]]])

In [ ]:
X

array([[[0.24615385],
        [0.23076923],
        [0.23076923],
        ...,
        [0.33846154],
        [0.16923077],
        [0.44615385]],

       [[0.23076923],
        [0.23076923],
        [0.33846154],
        ...,
        [0.16923077],
        [0.44615385],
        [0.44615385]],

       [[0.23076923],
        [0.33846154],
        [0.        ],
        ...,
        [0.44615385],
        [0.44615385],
        [0.        ]],

       ...,

       [[0.43076923],
        [0.16923077],
        [0.53846154],
        ...,
        [0.        ],
        [0.24615385],
        [0.16923077]],

       [[0.16923077],
        [0.53846154],
        [0.23076923],
        ...,
        [0.24615385],
        [0.16923077],
        [0.29230769]],

       [[0.53846154],
        [0.23076923],
        [0.43076923],
        ...,
        [0.16923077],
        [0.29230769],
        [0.46153846]]])

In [ ]:
X

array([[[0.24615385],
        [0.23076923],
        [0.23076923],
        ...,
        [0.33846154],
        [0.16923077],
        [0.44615385]],

       [[0.23076923],
        [0.23076923],
        [0.33846154],
        ...,
        [0.16923077],
        [0.44615385],
        [0.44615385]],

       [[0.23076923],
        [0.33846154],
        [0.        ],
        ...,
        [0.44615385],
        [0.44615385],
        [0.        ]],

       ...,

       [[0.43076923],
        [0.16923077],
        [0.53846154],
        ...,
        [0.        ],
        [0.24615385],
        [0.16923077]],

       [[0.16923077],
        [0.53846154],
        [0.23076923],
        ...,
        [0.24615385],
        [0.16923077],
        [0.29230769]],

       [[0.53846154],
        [0.23076923],
        [0.43076923],
        ...,
        [0.16923077],
        [0.29230769],
        [0.46153846]]])

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['msle'])

In [ ]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
from wandb.keras import WandbCallback

wandb.init(project='SeqGAN-phase1',
           entity=""            config={
                "learning_rate":0.002,
                "epochs": 4,
                "batch_size":256,
                "architecture": "GAN",
                "dataset":"dffe.txt",
            })
config =wandb.config

wandb: Currently logged in as: lingan007 (use `wandb login --relogin` to force relogin)
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [ ]:
model.fit(X, y, epochs=1, batch_size=256, callbacks=[WandbCallback()])

567/567 [==============================] - 4644s 8s/step - loss: 3.1108 - msle: 0.0070


In [ ]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['msle'])

OSError: Unable to open file (unable to open file: name = 'model_weights_saved.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

In [ ]:
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6e83c6a3-af48-46fc-acc2-fa28380e588c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>